<img src='filmes_natal.jpg'>

# Problema

Obter uma lista de filmes sobre o tema Natal no site iMDb e, para cada filme, as seguintes informações:

* Título
* link iMDb com mais informações sobre o filme
* Sinópse
* Diretor
* Gênero
* País
* Idioma
* Tempo de duração em minutos
* Atores principais
* Score
* Ano
* Popularidade
* Quantidade de reviews de usuários iMDb

Antes de atacarmos esse problema, precisamos entender algumas coisas básicas.

# Componentes de uma página web

Quando visitamos uma página, nosso navegador faz uma solicitação à um servidor de Internet. Essa solicitação é chamada de solicitação GET, pois estamos recebendo arquivos do servidor. O servidor então envia arquivos que dizem ao navegador como renderizar a página para nós. Os arquivos se enquadram em alguns tipos principais:

* HTML: Contém o conteúdo principal da página;
* CSS: Adiciona estilos para que a página fique mais bonita;
* JS: Arquivos JavaScript adicionam interatividade à página;
* Imagens: formatos de imagem, tais como JPG e PNG permitem que páginas web exibam imagens.

Após nosso navegador receber todos os arquivos, ele renderiza a página e a exibe para nós. Existem muitas coisas que acontecem no background para renderizar bem uma página, mas não precisamos nos preocupar com a maioria delas quando estamos realizando o web scraping. Quando realizamos o web scraping, estamos interessados no conteúdo principal da página, então olharemos para o HTML.

# Entendendo HTML

HyperText Markup Language (HTML) é uma linguagem com que as páginas web são criadas. O HTML não é uma linguagem de programação, como o Python – em vez disso, é uma linguagem de marcação que diz ao navegador como exibir o conteúdo. O HTML permite que você faça coisas similares ao Microsoft Word – criar parágrafos, deixar textos em negrito etc. Por não ser uma linguagem de programação, o HTML nem se aproxima da complexidade do Python.

O HTML é formado por elementos chamados tags. Veja uma exemplo de código em HTML:

```HTML
<html>
    <head>
    </head>
    <body>
        <p>
            Aqui um exemplo de parágrafo com texto.
            <a href="https://www.dataquest.io">Learn Data Science Online</a>
        </p>
        <p>
            Aqui um segundo exemplo de parágrafo com texto.
            <a href="https://www.python.org">Python</a>
        </p>
    </body>
</html>
```

No exemplo acima, temos duas tags a. As tags a são links, e dizem ao navegador para renderizar uma outra página. A propriedade href determina para onde o link vai. As tags a e p são extremamente comuns, a seguir temos outras:

* div – indica uma divisão, uma área na página;
* b – deixa qualquer texto dentro dela em negrito;
* i – deixa em itálico;
* table – cria uma tabela;
* form – cria um formulário;

Para uma lista completa de tags, clique [aqui](https://developer.mozilla.org/pt-BR/docs/Web/HTML/HTML5/HTML5_element_list).

# A biblioteca requests

A primeira coisa que precisamos fazer para realizar o web scrap é baixar a página. Podemos baixar as páginas utilizando a biblioteca requests do Python. A biblioteca requests fará uma solicitação GET ao servidor, que fará o download dos conteúdos HTML da página solicitada para nós. Existem vários tipos de solicitação diferentes que podemos realizar utilizando a biblioteca requests – GET é apenas um deles.

# A Biblioteca BeautifulSoup

Depois de baixamos um documento HTML da página que desejamos usando o requests. Nós podemos utilizar a biblioteca BeautifulSoup para analisar esse documento e extrair o conteúdo das tags que desejamos.

# Importando as Bibliotecas

In [1]:
import requests # para se conectar a página web
from bs4 import BeautifulSoup # para extrair informações do html
import pandas as pd # para manusear os dados e salva-los em csv 

# Obtendo os títulos e links dos Filmes

In [2]:
# url que lista filmes sobre o tema natal 
url_fonte = 'http://www.imdb.com/find?q=Christmas&s=tt&ref_=fn_tt'

In [3]:
# obtendo conteúdo da página
con = requests.get(url_fonte)

In [4]:
# criando objeto BeautifulSoup 
soup = BeautifulSoup(con.content, "html.parser")

In [5]:
titulos_filmes = [div.find('a').contents[0] for div in soup.findAll('td', attrs={'class':'result_text'})]

In [6]:
links_filmes = ['http://www.imdb.com' + div.a['href'] for div in soup.findAll('td', attrs={'class':'result_text'})]

# Obtendo as informações de cada filme

### Sinópse

In [49]:
url_filme = 'http://www.imdb.com/title/tt4537842/?ref_=rvi_tt'

In [50]:
# obtendo conteúdo da página
con = requests.get(url_filme)

# criando objeto BeautifulSoup 
soup = BeautifulSoup(con.content, "html.parser")

In [10]:
# <div class="summary_text" itemprop="description">
sinopse =  soup.find('div', attrs={'class':'summary_text'})

In [11]:
print sinopse.text        


Add a Plot »
            


In [12]:
# Fazendo função que irá capturar essa informação para nós

def get_sinopse(soup):
    '''
    Função rebe um objeto BeautifulSoup
    chamado soup com o html da página do filme
    desejado e retorna uma string da sinópse do filme.
    Caso não haja uma sinópse, retorna None.
    '''
    sinopse =  soup.find('div', attrs={'class':'summary_text'})
    
    if sinopse.text:
        return sinopse.text
    else:
        return None

In [13]:
# testando função

url_filme_com_sinopse = 'http://www.imdb.com/title/tt0097958/?ref_=fn_tt_tt_4'
url_filme_sem_sinopse = 'http://www.imdb.com/title/tt1410207/?ref_=fn_tt_tt_1'

urls = [url_filme_com_sinopse, url_filme_sem_sinopse]

for url in urls:
    con = requests.get(url)
    soup = BeautifulSoup(con.content, "html.parser")
    print get_sinopse(soup)


                    The Griswold family's plans for a big family Christmas predictably turn into a big disaster.
            

Add a Plot »
            


### Diretor

In [14]:
# <span class="itemprop" itemprop="name">Jeremiah S. Chechik</span>
diretor =  soup.find('span', attrs={'class':'itemprop'}, itemprop='name')

In [15]:
print diretor.text

Sebastián Lelio


In [16]:
def get_diretor(soup):
    '''
    retorna uma lista com 
    os gêneros do filme
    '''
    diretor =  soup.find('span', attrs={'class':'itemprop'}, itemprop='name')
    if diretor.text:
        return diretor.text
    else:
        return None

### Gênero

In [17]:
# tags com as informações do Gênero dos filmes
'''
<div class="see-more inline canwrap" itemprop="genre">
            <h4 class="inline">Genres:</h4>
<a href="/genre/Comedy?ref_=tt_stry_gnr"> Comedy</a>
        </div>
'''

div =  soup.find('div', attrs={'class':'see-more inline canwrap'}, itemprop='genre')

In [18]:
generos = [a.text for a in div.find_all('a')]

In [19]:
generos

[u' Drama', u' Romance']

In [82]:
def get_generos(soup):
    div =  soup.find('div', attrs={'class':'see-more inline canwrap'}, itemprop='genre')
    if div != None:
        div =  soup.find('div', attrs={'class':'see-more inline canwrap'}, itemprop='genre')
        generos = [a.text for a in div.find_all('a')]
    else:
        return None

### País

In [21]:
'''
    <div class="txt-block">
    <h4 class="inline">Country:</h4>
        <a href="/search/title?country_of_origin=cl&amp;ref_=tt_dt_dt" itemprop="url">Chile</a>
              <span class="ghost">|</span>
        <a href="/search/title?country_of_origin=fr&amp;ref_=tt_dt_dt" itemprop="url">France</a>
'''


tags_paises = soup.select("a[href*=country_of_origin]")

In [22]:
tags_paises

[<a href="/search/title?country_of_origin=cl&amp;ref_=tt_dt_dt" itemprop="url">Chile</a>,
 <a href="/search/title?country_of_origin=fr&amp;ref_=tt_dt_dt" itemprop="url">France</a>]

In [23]:
len(tags_paises)

2

In [24]:
paises = [a.text for a in tags_paises]

In [25]:
paises

[u'Chile', u'France']

In [26]:
def get_paises(soup):
    tags_paises = soup.select("a[href*=country_of_origin]")
    if len(tags_paises):
        paises = [a.text for a in tags_paises]
        return paises
    else:
        return None

### Idioma

In [27]:
'''
<a href="/search/title?title_type=feature&amp;primary_language=es&amp;sort=moviemeter,
asc&amp;ref_=tt_dt_dt" itemprop="url">Spanish</a>
'''

tags_idioma = soup.select("a[href*=primary_language]")

In [28]:
tags_idioma

[<a href="/search/title?title_type=feature&amp;primary_language=es&amp;sort=moviemeter,asc&amp;ref_=tt_dt_dt" itemprop="url">Spanish</a>]

In [29]:
tags_idioma[0].text

u'Spanish'

In [200]:
def get_idioma(soup):
    tags_idioma = soup.select("a[href*=primary_language]")
    if len(tags_idioma):
        return tags_idioma[0].text
    else:
        None

In [201]:
get_idioma(soup)

### Tempo de duração em minutos

In [173]:
'''
<time itemprop="duration" datetime="PT99M">99 min</time>
'''

tempo = soup.findAll('time', itemprop="duration")

In [174]:
tempo

[<time datetime="PT120M" itemprop="duration">\n                        2h\n                    </time>,
 <time datetime="PT120M" itemprop="duration">120 min</time>]

In [161]:
t = tempo[0].text

In [162]:
t

u'\n                        50min\n                    '

In [163]:
t.find('h')

-1

In [158]:
t = t.replace(' ', '')

In [159]:
t

u'\n50min\n'

In [160]:
t = t.replace('\n', '')
t

u'50min'

In [35]:
t2 = t[:t.find(' min')]
t2

u'99'

In [36]:
int(t2)

99

In [148]:
duracao = '\n  1h 30min \n'

In [175]:
def transformar_horas_em_minutos(text):
    if text.find('h') != -1 and text.find('min') != -1:
        tempo = text.replace(' ', '')
        tempo = tempo.replace('\n', '')
        tempo = tempo.replace('min', '')
        h = int(tempo[0])*60
        m = int(tempo[2:])
        return h + m
    elif text.find('h') != -1 and text.find('min') == -1:
        tempo = text.replace(' ', '')
        tempo = tempo.replace('\n', '')
        tempo = tempo.replace('min', '')
        return int(tempo[0])*60
    else:
        tempo = text.replace(' ', '')
        tempo = tempo.replace('\n', '')
        tempo = tempo.replace('min', '')
        return int(tempo)

In [176]:
transformar_horas_em_minutos(duracao)

90

In [177]:
transformar_horas_em_minutos('\n  50min  \n')

50

In [178]:
def get_duracao(soup):
    tempo = soup.findAll('time', itemprop="duration")
    if len(tempo):
        return transformar_horas_em_minutos(tempo[0].text)   
    else:
        return None

In [179]:
get_duracao(soup)

120

### Atores Principais

In [38]:
'''
<span itemprop="actors" itemscope="" itemtype="http://schema.org/Person">
<a href="/name/nm1443757?ref_=tt_ov_st_sm" itemprop="url"><span class="itemprop" itemprop="name">Manuela Martelli</span></a>,             </span>
'''
atores = soup.findAll('span', itemprop="actors")

In [39]:
atores

[<span itemprop="actors" itemscope="" itemtype="http://schema.org/Person">\n<a href="/name/nm1443757?ref_=tt_ov_st_sm" itemprop="url"><span class="itemprop" itemprop="name">Manuela Martelli</span></a>,             </span>,
 <span itemprop="actors" itemscope="" itemtype="http://schema.org/Person">\n<a href="/name/nm2926981?ref_=tt_ov_st_sm" itemprop="url"><span class="itemprop" itemprop="name">Diego Ruiz</span></a>,             </span>,
 <span itemprop="actors" itemscope="" itemtype="http://schema.org/Person">\n<a href="/name/nm3396723?ref_=tt_ov_st_sm" itemprop="url"><span class="itemprop" itemprop="name">Alicia Rodr\xedguez</span></a> </span>]

In [40]:
for div in atores:
    print div.a.text

Manuela Martelli
Diego Ruiz
Alicia Rodríguez


In [41]:
def gets_atores(soup):
    divs_atores = soup.findAll('span', itemprop="actors")
    atores = [div.a.text for div in divs_atores]
    return atores

### Score

In [42]:
'''
<span itemprop="ratingValue">6,2</span>
'''

score = soup.find('span', itemprop="ratingValue")

In [43]:
float(score.text)

6.2

In [181]:
def get_score(soup):
    score = soup.find('span', itemprop="ratingValue")
    if score != None:
        return float(score.text)
    else:
        return None

### Ano

In [45]:
# <a href="/year/2009/?ref_=tt_ov_inf">2009</a>

ano = soup.select("a[href*=/year/]")

In [46]:
ano

[<a href="/year/2009/?ref_=tt_ov_inf">2009</a>]

In [47]:
ano[0].text

u'2009'

In [48]:
def get_ano(soup):
    ano = soup.select("a[href*=/year/]")
    return int(ano[0].text)

### Popularidade

In [51]:
'''
<span class="popularityUpOrFlat">362</span>
'''

popularidade = soup.find('span', attrs={'class': 'popularityUpOrFlat'})

In [53]:
popularidade.text

u'362'

In [94]:
int(popularidade.text)

362

In [95]:
int('1,151')

ValueError: invalid literal for int() with base 10: '1,151'

In [102]:
def tirar_virgula(text):
    if text.find(',') != -1:
        text = list(text)
        text.remove(',')
        return ''.join(text)
    else:
        return text

In [105]:
tirar_virgula('362')

'362'

In [106]:
tirar_virgula('1,151')

'1151'

In [107]:
def get_popularidade(soup):
    popularidade = soup.find('span', attrs={'class': 'popularityUpOrFlat'})
    if popularidade != None:
        return int(tirar_virgula(popularidade.text))
    else:
        return None

### Quantidade de reviews de usuários iMDb

In [183]:
'''
<a href="reviews?ref_=tt_ov_rt">55 user</a>
'''

reviews = soup.select("a[href*=reviews?ref_=tt_ov_rt]")

In [184]:
reviews

[<a href="externalreviews?ref_=tt_ov_rt">2 critic</a>]

In [190]:
r = reviews[0].text

In [192]:
r.find('user')

-1

In [63]:
qtd_r = r.text
qtd_r

u'55 user'

In [64]:
qtd_r = int(qtd_r[:qtd_r.find(' user')])
qtd_r

55

In [197]:
def get_qtdReviews(soup):
    reviews = soup.select("a[href*=reviews?ref_=tt_ov_rt]")
    if len(reviews):
        if reviews[0].text.find('user') != -1:
            r = reviews[0]
            qtd_r = r.text
            return int(qtd_r[:qtd_r.find(' user')])
        else:
            return None
    
    else:
        return None

In [198]:
get_qtdReviews(soup)

# Obtendo os Dados para Todos os Filmes

Cada filme será um dicionário do tipo:

In [68]:
filme = {'titulo': 'string','link': 'string', 'sinopse': 'string', 'diretor': 'string','genero': 'lista de string',
         'pais': 'lista de string', 'Idioma': 'lista de string','duracao': 'int', 'atores': 'lista de string',
         'score': 'float', 'ano': 'int','popularidade': 'int','qtd_reviews':'int'}

Teremos então uma lista de dicionários filmes.

In [70]:
len(titulos_filmes)

200

In [202]:
lista_filmes = []
for titulo, link in zip(titulos_filmes, links_filmes):
    dic = {}
    dic['titulo'] = titulo
    dic['link'] = link
    
    con = requests.get(link)
    soup = BeautifulSoup(con.content, "html.parser")
    con.close()
    
    dic['sinopse'] = get_sinopse(soup)
    dic['diretor'] = get_diretor(soup)
    dic['genero'] = get_generos(soup)
    dic['pais'] = get_paises(soup)
    dic['idioma'] = get_idioma(soup)
    dic['duracao'] = get_duracao(soup)
    dic['atores'] = gets_atores(soup)
    dic['score'] = get_score(soup)
    dic['ano'] = get_ano(soup)
    dic['popularidade'] = get_popularidade(soup)
    dic['qtd_reviews'] = get_qtdReviews(soup)
    
    lista_filmes.append(dic)

# Análieses dos Dados

### Nuvem de Palavras das sinópses

In [217]:
todas_sinopses = ''
for dic in lista_filmes:
    todas_sinopses += dic['sinopse']

In [218]:
f = open('sinopses.txt', 'w')
f.write(todas_sinopses.encode('utf-8'))
f.close()

<img src='wordcload_sinopses.png'>